## STRAICO

In [ ]:
import json
import csv
import os
import requests
from typing import List, Optional, Dict, Any
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from functools import partial
import time
from datetime import datetime, timedelta

# API endpoint
API_URL = "https://api.straico.com/v1/prompt/completion"

# Hardcoded API key and default model(s)
API_KEY = "TA-bB4gKjQn7D4NcB6ycoj1YaACCfTjLOFEVXaOzarATGSm4LUD"

PROMPT_ID_MAP = {
    "prompt_1": "P1", # Dada la palabra --> Definición del modismo.
    "prompt_2": "P2", # Dada la palabra --> Decir si es modismo o no.
    "prompt_3": "P3", # Dado el ejemplo --> Reemplazo literal y definición del remplazo.
}

# Configuración de paralelización
MAX_WORKERS = 8

# Configuración de guardado incremental
SAVE_EVERY_N_ITEMS = 100 # Guardar cada 100 items procesados por modelo

# Lock global para operaciones de escritura thread-safe
_write_lock = threading.Lock()

In [ ]:
def send_prompt(message: str, models: Optional[List[str]] = None) -> Dict[str, Any]:
    
    """ Send a single text prompt to Straico and return the parsed response.
    This function always performs a live HTTP request.
    """

    payload: Dict[str, Any] = {"models": models, "message": message}
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

    try:
        resp = requests.post(API_URL, headers=headers, json=payload, timeout=120)
    except requests.exceptions.RequestException as exc:
        return {"error": str(exc)}

    # Parse JSON if available
    try:
        data = resp.json()
    except Exception:
        data = None

    if 200 <= resp.status_code < 300:

        # Try to extract the assistant 'content' text from the common response
        # shape: data -> completions -> <model> -> completion -> choices[0] -> message -> content

        if isinstance(data, dict):
            completions = data.get('data', {}).get('completions', {})

            # If we passed a single model, try to use that key; otherwise take first
            model_key = None

            if models and len(models) == 1:
                model_key = models[0]
            if not model_key and isinstance(completions, dict) and len(completions) > 0:
                model_key = next(iter(completions.keys()))

            if model_key and model_key in completions:
                try:
                    content = completions[model_key]['completion']['choices'][0]['message']['content']
                    return content
                
                except Exception:
                    pass

        # Fallback: return raw text or the full JSON string
        if data is None:
            return resp.text
        
        try:
            return json.dumps(data, ensure_ascii=False)
        except Exception:
            return str(data)

    return {"error": f"status={resp.status_code}", "response": data if data is not None else resp.text}

### Configuración General

In [ ]:
def _import_prompts() -> Dict[str, str]:
    """Load prompt_1..prompt_3 from prompts.py and return as a dict.
    If prompts.py isn't present, returns an empty dict.
    """
    try:
        from Straico import prompts as p # type: ignore
    except Exception:
        try:
            import prompts as p # type: ignore
        except Exception:
            return {}

    out: Dict[str, str] = {}
    for name in ("prompt_1", "prompt_2", "prompt_3"):
        if hasattr(p, name):
            out[name] = getattr(p, name)
    return out

#### Modelos a usar

In [ ]:
# Cargar modelos desde el archivo text_model_ids.txt
MODELS_FILE = 'Straico/text_model_usefull.txt'

def load_models_from_file(filepath):
    """Carga los nombres de modelos desde un archivo de texto."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo {filepath}")
        return []

DEFAULT_MODELS = load_models_from_file(MODELS_FILE)

In [ ]:
DEFAULT_MODELS

#### Ejecusión

In [ ]:
DATASET_PATH = 'modismos_Dataset_Cleaned.csv'
RESPONSES_DIR = 'Straico'

# Calcular número total de filas en el dataset
with open(DATASET_PATH, encoding='utf-8') as f:
 TOTAL_DATASET_ROWS = sum(1 for _ in csv.DictReader(f, delimiter=';'))

print(f"Dataset: {DATASET_PATH}")
print(f"Total de modismos disponibles: {TOTAL_DATASET_ROWS:,}")

# Cargar prompts desde prompts.py
PROMPTS = _import_prompts()
print(f"Prompts cargados: {', '.join(PROMPTS.keys())}")

In [ ]:
# Configurar el número de filas a procesar
# N_ROWS = 1 # Para pruebas
N_ROWS = None # Para procesar todo el dataset (4617 modismos)

In [ ]:
def cargar_dataset(n_rows=None):
    """Carga el dataset y retorna una lista de diccionarios con modismo, significado y ejemplo."""
    rows = []
    seen_modismos = set()
    
    with open(DATASET_PATH, encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=';')
        for r in reader:
            modismo = r.get('modismo', '').strip()
            if not modismo or modismo.casefold() in seen_modismos:
                continue
            seen_modismos.add(modismo.casefold())
            
            rows.append({
                'modismo': modismo,
                'significado': r.get('significado', '').strip(),
                'ejemplo': r.get('ejemplo', '').strip()
            })
            
            if n_rows and len(rows) >= n_rows:
                break
    
    return rows


def sanitize_model_name(model_name):
    """Convierte nombres de modelos en nombres válidos para nombres de archivos."""
    return model_name.replace('/', '_').replace(':', '_').replace('-', '_').replace('.', '_')


def save_json(filepath, data):
    """Guarda datos en un archivo JSON de forma thread-safe."""
    with _write_lock:
        try:
            dir_path = os.path.dirname(filepath)
            if dir_path:
                os.makedirs(dir_path, exist_ok=True)
            
            # Escritura atómica usando archivo temporal
            temp_filepath = f"{filepath}.tmp"
            with open(temp_filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            
            # Renombrar es operación atómica en sistemas POSIX
            os.replace(temp_filepath, filepath)
        except Exception as e:
            print(f"[ERROR] No se pudo guardar {filepath}: {e}")
            # Limpiar archivo temporal si existe
            if os.path.exists(temp_filepath):
                try:
                    os.remove(temp_filepath)
                except:
                    pass


def load_checkpoint(base_dir, prompt_name, model_name):
    """Carga el checkpoint de un modelo si existe."""
    model_safe_name = sanitize_model_name(model_name)
    checkpoint_path = os.path.join(base_dir, prompt_name, model_safe_name, "checkpoint.json")
    
    if os.path.exists(checkpoint_path):
        try:
            with open(checkpoint_path, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return None
    return None


def save_checkpoint(base_dir, prompt_name, model_name, checkpoint_data):
    """Guarda el checkpoint de un modelo."""
    model_safe_name = sanitize_model_name(model_name)
    checkpoint_path = os.path.join(base_dir, prompt_name, model_safe_name, "checkpoint.json")
    save_json(checkpoint_path, checkpoint_data)


def save_model_response(base_dir, prompt_name, model_name, data):
    """Guarda la respuesta de un modelo específico en su propia carpeta."""
    model_safe_name = sanitize_model_name(model_name)
    model_dir = os.path.join(base_dir, prompt_name, model_safe_name)
    filepath = os.path.join(model_dir, f"{model_safe_name}.json")
    save_json(filepath, data)


def save_consolidated_response(base_dir, prompt_name, all_models_data):
    """Guarda todas las respuestas consolidadas en un solo archivo."""
    filepath = os.path.join(base_dir, prompt_name, "all_models.json")
    save_json(filepath, all_models_data)


def is_model_completed(base_dir, prompt_name, model_name, expected_items):
    """Verifica si un modelo ya completó todo el procesamiento."""
    model_safe_name = sanitize_model_name(model_name)
    model_file = os.path.join(base_dir, prompt_name, model_safe_name, f"{model_safe_name}.json")
    
    # Si existe checkpoint, significa que no ha terminado
    checkpoint = load_checkpoint(base_dir, prompt_name, model_name)
    if checkpoint:
        return False
    
    # Verificar si existe el archivo final y tiene todos los items
    if os.path.exists(model_file):
        try:
            with open(model_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                return len(data) >= expected_items
        except:
            return False
    
    return False


def format_time(seconds):
    """Formatea segundos a formato legible."""
    if seconds < 60:
        return f"{seconds:.1f}s"
    elif seconds < 3600:
        minutes = seconds / 60
        return f"{minutes:.1f}m"
    else:
        hours = seconds / 3600
        return f"{hours:.1f}h"


def print_progress_bar(current, total, prefix='', suffix='', length=40, fill='█'):
    """Imprime una barra de progreso elegante."""
    percent = 100 * (current / float(total))
    filled_length = int(length * current // total)
    bar = fill * filled_length + '─' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent:.1f}% {suffix}', end='', flush=True)
    if current == total:
        print() # Nueva línea al completar

### Utilidades de Gestión

In [ ]:
def check_checkpoints(prompt_name, models=DEFAULT_MODELS):
    """Verifica el estado de los checkpoints para un prompt específico."""
    print(f"Verificando checkpoints para {prompt_name}...\n")
    
    has_checkpoints = False
    for model in models:
        checkpoint = load_checkpoint(RESPONSES_DIR, prompt_name, model)
        if checkpoint:
            has_checkpoints = True
            items = checkpoint.get('items_completed', 0)
            errors = checkpoint.get('errors_count', 0)
            last_updated = checkpoint.get('last_updated', 'desconocido')
            print(f"  [OK] {model}")
            print(f"       Items completados: {items}")
            print(f"       Errores: {errors}")
            print(f"       Última actualización: {last_updated}")
            print()
    
    if not has_checkpoints:
        print("  [INFO] No hay checkpoints pendientes")
    
    return has_checkpoints


def clear_checkpoints(prompt_name, models=DEFAULT_MODELS, confirm=True):
    """Limpia todos los checkpoints para un prompt específico."""
    if confirm:
        response = input(f"[ADVERTENCIA] ¿Estas seguro de eliminar todos los checkpoints de {prompt_name}? (si/no): ")
        if response.lower() not in ['si', 'sí', 's', 'yes', 'y']:
            print("[CANCELADO] Operacion cancelada")
            return
    
    deleted = 0
    for model in models:
        model_safe_name = sanitize_model_name(model)
        checkpoint_path = os.path.join(RESPONSES_DIR, prompt_name, model_safe_name, "checkpoint.json")
        if os.path.exists(checkpoint_path):
            try:
                os.remove(checkpoint_path)
                deleted += 1
            except Exception as e:
                print(f"  [ERROR] Error eliminando checkpoint de {model}: {e}")
    
    print(f"[OK] {deleted} checkpoint(s) eliminado(s)")


def get_processing_stats(prompt_name):
    """Obtiene estadísticas de procesamiento para un prompt."""
    print(f"Estadisticas de {prompt_name}\n")
    
    consolidated_path = os.path.join(RESPONSES_DIR, prompt_name, "all_models.json")
    
    if not os.path.exists(consolidated_path):
        print("  [INFO] No hay datos procesados aun")
        return
    
    with open(consolidated_path, 'r', encoding='utf-8') as f:
        all_data = json.load(f)
    
    total_items = 0
    total_errors = 0
    
    print(f"{'Modelo':<40} {'Items':<10} {'Errores'}")
    print("─" * 65)
    
    for model, responses in all_data.items():
        items = len(responses)
        errors = sum(1 for r in responses if isinstance(r.get('response'), dict) and 'error' in r['response'])
        total_items += items
        total_errors += errors
        print(f"{model:<40} {items:<10} {errors}")
    
    print("─" * 65)
    print(f"{'TOTAL':<40} {total_items:<10} {total_errors}")
    print()
    print(f"Modelos procesados: {len(all_data)}")
    print(f"Promedio items/modelo: {total_items/len(all_data):.1f}" if all_data else "N/A")

In [ ]:
# Ejemplos de uso de utilidades:

# Ver checkpoints pendientes para Prompt 1
check_checkpoints("Prompt 2")

# Ver estadísticas de procesamiento
# get_processing_stats("Prompt 1")

# Limpiar checkpoints (¡cuidado! esto reiniciará el progreso)
# clear_checkpoints("Prompt 1")

print("[INFO] Descomenta las lineas arriba para usar las utilidades")

## PROMPT 2: Modismo → Es Modismo (Sí/No)

In [ ]:
class ProgressTracker:
    """Clase para rastrear el progreso de múltiples modelos en paralelo."""
    
    def __init__(self, total_models, total_items_per_model):
        self.total_models = total_models
        self.total_items_per_model = total_items_per_model
        self.model_progress = {} # {model_name: items_completed}
        self.model_errors = {} # {model_name: error_count}
        self.completed_models = 0
        self.start_time = time.time()
        self.lock = threading.Lock()
    
    def update_model_progress(self, model_name, items_completed, error_count=0):
        """Actualiza el progreso de un modelo específico."""
        with self.lock:
            self.model_progress[model_name] = items_completed
            self.model_errors[model_name] = error_count
    
    def mark_model_completed(self, model_name):
        """Marca un modelo como completado."""
        with self.lock:
            self.completed_models += 1
    
    def get_summary(self):
        """Retorna un resumen del progreso actual."""
        with self.lock:
            total_items = sum(self.model_progress.values())
            total_errors = sum(self.model_errors.values())
            elapsed = time.time() - self.start_time
            
            total_expected = self.total_models * self.total_items_per_model
            progress_pct = (total_items / total_expected * 100) if total_expected > 0 else 0
            
            # Estimación de tiempo restante
            if total_items > 0:
                items_per_sec = total_items / elapsed
                items_remaining = total_expected - total_items
                eta_seconds = items_remaining / items_per_sec if items_per_sec > 0 else 0
            else:
                eta_seconds = 0
            
            return {
                'completed_models': self.completed_models,
                'total_models': self.total_models,
                'total_items': total_items,
                'total_expected': total_expected,
                'total_errors': total_errors,
                'progress_pct': progress_pct,
                'elapsed': elapsed,
                'eta': eta_seconds
            }
    
    def print_status(self):
        """Imprime el estado actual de forma elegante."""
        summary = self.get_summary()
        
        # Barra de progreso
        bar_length = 40
        filled = int(bar_length * summary['progress_pct'] / 100)
        bar = '█' * filled + '─' * (bar_length - filled)
        
        # Formato de salida
        print(f"\r{'':100}", end='') # Limpiar línea
        status = (
            f"\r[{bar}] {summary['progress_pct']:.1f}% | "
            f"Modelos: {summary['completed_models']}/{summary['total_models']} | "
            f"Items: {summary['total_items']}/{summary['total_expected']} | "
            f"Errores: {summary['total_errors']} | "
            f"Tiempo: {format_time(summary['elapsed'])} | "
            f"ETA: {format_time(summary['eta'])}"
        )
        print(status, end='', flush=True)


def process_single_model_prompt_1(model, dataset, template, responses_dir="Straico", progress_tracker=None):
    """
    Procesa un solo modelo para el Prompt 1 con guardado incremental y checkpoint.
    """
    model_safe_name = sanitize_model_name(model)
    
    # Verificar si el modelo ya completó todo el procesamiento
    if is_model_completed(responses_dir, "Prompt 1", model, len(dataset)):
        print(f"\n[SKIP] {model} - Ya completado anteriormente")
        if progress_tracker:
            # Cargar datos existentes para actualizar el tracker
            model_file = os.path.join(responses_dir, "Prompt 1", model_safe_name, f"{model_safe_name}.json")
            try:
                with open(model_file, 'r', encoding='utf-8') as f:
                    existing_data = json.load(f)
                    errors = sum(1 for r in existing_data if isinstance(r.get('response'), dict) and 'error' in r['response'])
                    progress_tracker.update_model_progress(model, len(existing_data), errors)
                    progress_tracker.mark_model_completed(model)
                    return model, existing_data, errors
            except:
                pass
        return model, [], 0
    
    # Cargar checkpoint si existe
    checkpoint = load_checkpoint(responses_dir, "Prompt 1", model)
    if checkpoint:
        model_responses = checkpoint.get('responses', [])
        processed_modismos = set(checkpoint.get('processed_modismos', []))
        start_idx = len(model_responses)
        print(f"\n[CHECKPOINT] {model} - Reanudando desde item {start_idx}")
    else:
        model_responses = []
        processed_modismos = set()
        start_idx = 0
    
    errors_count = 0
    total = len(dataset)
    
    for idx, row in enumerate(dataset, 1):
        try:
            modismo = row.get('modismo', '').strip()
            if not modismo or modismo in processed_modismos:
                continue
            
            # Armar el prompt
            prompt_text = template.replace('{{modismo}}', modismo)
            
            # Obtener respuesta del modelo con retry mejorado
            max_retries = 3
            resp = None
            for attempt in range(max_retries):
                try:
                    resp = send_prompt(prompt_text, models=[model])
                    if not isinstance(resp, dict) or 'error' not in resp:
                        break
                except Exception as e:
                    if attempt == max_retries - 1:
                        resp = {"error": str(e)}
                        break
                
                # Backoff exponencial con jitter
                wait_time = (2 ** attempt) + (time.time() % 1)
                time.sleep(wait_time)
            
            # Procesar respuesta
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    response_data = parsed
                except:
                    response_data = {"raw_response": resp}
            elif isinstance(resp, dict):
                if 'error' in resp:
                    errors_count += 1
                response_data = resp
            else:
                response_data = {"raw_response": str(resp)}
            
            # Agregar metadatos
            entry = {
                "modismo": modismo,
                "model": model,
                "response": response_data
            }
            
            model_responses.append(entry)
            processed_modismos.add(modismo)
            
            # Actualizar progreso
            if progress_tracker:
                progress_tracker.update_model_progress(model, len(model_responses), errors_count)
                progress_tracker.print_status()
            
            # Guardado incremental cada N items
            if len(model_responses) % SAVE_EVERY_N_ITEMS == 0:
                checkpoint_data = {
                    'responses': model_responses,
                    'processed_modismos': list(processed_modismos),
                    'last_updated': datetime.now().isoformat(),
                    'items_completed': len(model_responses),
                    'errors_count': errors_count
                }
                save_checkpoint(responses_dir, "Prompt 1", model, checkpoint_data)
                save_model_response(responses_dir, "Prompt 1", model, model_responses)
            
        except Exception as e:
            errors_count += 1
            continue
    
    # Guardar respuestas finales del modelo
    try:
        save_model_response(responses_dir, "Prompt 1", model, model_responses)
        # Limpiar checkpoint
        model_safe_name = sanitize_model_name(model)
        checkpoint_path = os.path.join(responses_dir, "Prompt 1", model_safe_name, "checkpoint.json")
        if os.path.exists(checkpoint_path):
            os.remove(checkpoint_path)
    except Exception as e:
        print(f"\n[ERROR] {model} - No se pudo guardar: {e}")
    
    if progress_tracker:
        progress_tracker.mark_model_completed(model)
    
    return model, model_responses, errors_count





def process_single_model_prompt_2(model, dataset, template, responses_dir="Straico", progress_tracker=None):
    """
    Procesa un solo modelo para el Prompt 2 con guardado incremental y checkpoint.
    """
    model_safe_name = sanitize_model_name(model)
    
    # Verificar si el modelo ya completó todo el procesamiento
    if is_model_completed(responses_dir, "Prompt 2", model, len(dataset)):
        print(f"[SKIP] {model} - Ya completado anteriormente")
        if progress_tracker:
            # Cargar datos existentes para actualizar el tracker
            model_file = os.path.join(responses_dir, "Prompt 2", model_safe_name, f"{model_safe_name}.json")
            try:
                with open(model_file, 'r', encoding='utf-8') as f:
                    existing_data = json.load(f)
                    errors = sum(1 for r in existing_data if isinstance(r.get('response'), dict) and 'error' in r['response'])
                    progress_tracker.update_model_progress(model, len(existing_data), errors)
                    progress_tracker.mark_model_completed(model)
                    return model, existing_data, errors
            except:
                pass
        return model, [], 0
    
    # Cargar checkpoint si existe
    checkpoint = load_checkpoint(responses_dir, "Prompt 2", model)
    if checkpoint:
        model_responses = checkpoint.get('responses', [])
        processed_modismos = set(checkpoint.get('processed_modismos', []))
        start_idx = len(model_responses)
        print(f"[CHECKPOINT] {model} - Reanudando desde item {start_idx}")
    else:
        model_responses = []
        processed_modismos = set()
        start_idx = 0
    
    errors_count = 0
    total = len(dataset)
    
    for idx, row in enumerate(dataset, 1):
        try:
            modismo = row.get('modismo', '').strip()
            if not modismo or modismo in processed_modismos:
                continue
            
            # Armar el prompt
            prompt_text = template.replace('{{modismo}}', modismo)
            
            # Obtener respuesta del modelo con retry mejorado
            max_retries = 3
            resp = None
            for attempt in range(max_retries):
                try:
                    resp = send_prompt(prompt_text, models=[model])
                    if not isinstance(resp, dict) or 'error' not in resp:
                        break
                except Exception as e:
                    if attempt == max_retries - 1:
                        resp = {"error": str(e)}
                        break
                
                wait_time = (2 ** attempt) + (time.time() % 1)
                time.sleep(wait_time)
            
            # Procesar respuesta
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    response_data = parsed
                except:
                    response_data = {"raw_response": resp}
            elif isinstance(resp, dict):
                if 'error' in resp:
                    errors_count += 1
                response_data = resp
            else:
                response_data = {"raw_response": str(resp)}
            
            # Agregar metadatos
            entry = {
                "modismo": modismo,
                "model": model,
                "response": response_data
            }
            
            model_responses.append(entry)
            processed_modismos.add(modismo)
            
            # Actualizar progreso
            if progress_tracker:
                progress_tracker.update_model_progress(model, len(model_responses), errors_count)
                progress_tracker.print_status()
            
            # Guardado incremental cada N items
            if len(model_responses) % SAVE_EVERY_N_ITEMS == 0:
                checkpoint_data = {
                    'responses': model_responses,
                    'processed_modismos': list(processed_modismos),
                    'last_updated': datetime.now().isoformat(),
                    'items_completed': len(model_responses),
                    'errors_count': errors_count
                }
                save_checkpoint(responses_dir, "Prompt 2", model, checkpoint_data)
                save_model_response(responses_dir, "Prompt 2", model, model_responses)
            
        except Exception as e:
            errors_count += 1
            continue
    
    # Guardar respuestas finales del modelo
    try:
        save_model_response(responses_dir, "Prompt 2", model, model_responses)
        # Limpiar checkpoint
        model_safe_name = sanitize_model_name(model)
        checkpoint_path = os.path.join(responses_dir, "Prompt 2", model_safe_name, "checkpoint.json")
        if os.path.exists(checkpoint_path):
            os.remove(checkpoint_path)
    except Exception as e:
        print(f"\n[ERROR] {model} - No se pudo guardar: {e}")
    
    if progress_tracker:
        progress_tracker.mark_model_completed(model)
    
    return model, model_responses, errors_count


def run_prompt_2(models=DEFAULT_MODELS, n_rows=N_ROWS, max_workers=MAX_WORKERS):
    """
    PROMPT 2: Dada la palabra/modismo -> Determinar si es modismo (Si/No) (PARALELIZADO con progreso)
    INPUT: modismo
    OUTPUT: es_modismo
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 2: Modismo -> Es Modismo (Si/No) (PARALELO)")
    print("=" * 80)

    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    if not dataset:
        print("[ERROR] No se pudo cargar el dataset")
        return
    
    print(f"\nConfiguracion:")
    print(f"  Dataset: {len(dataset)} modismos")
    print(f"  Modelos: {len(models)}")
    print(f"  Workers paralelos: {max_workers}")
    print(f"  Guardado incremental: cada {SAVE_EVERY_N_ITEMS} items")
    print(f"  Total de peticiones: {len(dataset) * len(models):,}")

    # Obtener template del prompt
    template = PROMPTS.get('prompt_2')
    if not template:
        print("[ERROR] prompt_2 no encontrado")
        return

    # Inicializar tracker de progreso
    progress_tracker = ProgressTracker(len(models), len(dataset))
    
    all_models_data = {}
    total_errors = 0
    
    print("\n" + "─" * 80)
    print("Iniciando procesamiento...")
    print("─" * 80)
    
    start_time = time.time()
    
    try:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            process_func = partial(
                process_single_model_prompt_2, 
                dataset=dataset, 
                template=template, 
                responses_dir=RESPONSES_DIR,
                progress_tracker=progress_tracker
            )
            
            future_to_model = {executor.submit(process_func, model): model for model in models}
            
            for future in as_completed(future_to_model):
                model = future_to_model[future]
                try:
                    model_name, model_responses, errors = future.result(timeout=3600)
                    all_models_data[model_name] = model_responses
                    total_errors += errors
                except Exception as exc:
                    print(f"\n[ERROR] {model} - FALLO CRÍTICO: {exc}")
                    all_models_data[model] = []
                    progress_tracker.mark_model_completed(model)
                    progress_tracker.print_status()
    
    except KeyboardInterrupt:
        print("\n\n[ADVERTENCIA] INTERRUMPIDO por el usuario")
        summary = progress_tracker.get_summary()
        print(f"Progreso guardado: {summary['completed_models']}/{summary['total_models']} modelos")
        return

    elapsed_time = time.time() - start_time
    
    print("\n\n" + "─" * 80)
    print("  Guardando archivo consolidado...")
    try:
        save_consolidated_response(RESPONSES_DIR, "Prompt 2", all_models_data)
        print("[OK] Archivo consolidado guardado")
    except Exception as e:
        print(f"[ERROR] No se pudo guardar consolidado: {e}")

    # Resumen final
    summary = progress_tracker.get_summary()
    print("\n" + "=" * 80)
    print("PROMPT 2 - COMPLETADO")
    print("=" * 80)
    print(f"Estadisticas:")
    print(f"  Modelos procesados: {summary['completed_models']}/{summary['total_models']}")
    print(f"  Items procesados: {summary['total_items']:,}/{summary['total_expected']:,}")
    print(f"  Errores totales: {total_errors:,}")
    print(f"  Tiempo total: {format_time(elapsed_time)}")
    print(f"  Tiempo promedio: {format_time(elapsed_time/len(models))}/modelo")
    
    if summary['total_items'] > 0:
        avg_time_per_item = elapsed_time / summary['total_items']
        print(f"  Velocidad: {format_time(avg_time_per_item)}/item")
    
    print("=" * 80)

In [ ]:
# EJECUTAR PROMPT 2
# Procesa todos los modelos en paralelo con progreso en tiempo real
# Si se interrumpe (Ctrl+C), puede reanudar desde los checkpoints

run_prompt_2()